# AC290 Extreme Computing: Project-based High Performance Distributed and Parallel Systems

## Module 1: Influence Maximization Problem

<img src='img/AC290_IMP_graph1.png' style="width: 500px;">
<p style="text-align: center;">Yelp Reviewer Network</p>

In [34]:
%matplotlib inline
import time
import numpy as np
import matplotlib.pyplot as plt

## 1. Python

### 1.1. Time Profiling

We'll examine two different ways to profile run time of a function by using the following function below:

An example from http://www.huyng.com/posts/python-performance-analysis/

In [35]:
def primes(n): 
    if n == 2:
        return [2]
    elif n < 2:
        return []
    s = range(3,n+1,2)
    mroot = n ** 0.5
    half = (n + 1) / 2 - 1
    i = 0; m = 3;
    while m <= mroot:
        if s[i]:
            j = (m * m-3) / 2
            s[j] = 0
            while j < half:
                s[j] = 0
                j += m
        i = i + 1
        m = 2 * i + 3
    return [2]+[x for x in s if x]

The simplest method to evaluate your function's run time is to use Python's time module and manually print out the run time of your function.  This is a quick and dirty way to measure your function's performance.

In [36]:
import time

In [37]:
start = time.time()
prime_nums = primes(int(10e6))
duration = time.time() - start
print "%f Seconds" % duration

2.568427 Seconds


Using this approach, we can store the run time of the your function for analysis later on (mean, standard deviation, etc.). 

Expanding on this simple method, let's create a function decorator that will automate this process every time we want to evaluate a function's time performance.

In [38]:
from functools import wraps

def timefn(func):
    @wraps(func)
    def calc_time(*args, **kwargs):
        t1=time.time()
        result = func(*args,**kwargs)
        t2=time.time()
        print "@timefn: %.5f Seconds" % (t2-t1)
        return result
    return calc_time

In [39]:
@timefn
def primes(n): 
    if n == 2:
        return [2]
    elif n < 2:
        return []
    s = range(3,n+1,2)
    mroot = n ** 0.5
    half = (n + 1) / 2 - 1
    i = 0; m = 3;
    while m <= mroot:
        if s[i]:
            j = (m * m-3) / 2
            s[j] = 0
            while j < half:
                s[j] = 0
                j += m
        i = i + 1
        m = 2 * i + 3
    return [2]+[x for x in s if x]

In [40]:
prime_nums = primes(int(10e6))

@timefn: 2.43784 Seconds


Another simple method is to use iPython's %time magic function.

In [41]:
%time prime_nums = primes(int(10e6))

@timefn: 2.88802 Seconds
CPU times: user 2.85 s, sys: 39.4 ms, total: 2.88 s
Wall time: 2.91 s


### 1.2. Set vs List vs Tuple

>"One of the most important things in writing efficient programs is understanding the guarantees of the data structures you use.  In fact, a large part of performant programming is understanding what questions you are trying to ask of your data and picking a data structure that you can answer these questions quickly." - Gorelick & Ozsvald

Python provides 4 basic built-in data types - list, set, tuple, and dictionary (set without values).  We will compare the run time performance of each data types for different set of problems in order to understand which data structure provides the best solution to each unique problem.

In [42]:
n = int(10e6)

In [43]:
python_list = list(xrange(n))
python_set = set(xrange(n))
python_tuple = tuple(xrange(n))

### 1.2.1. Membership

List - dynamic array

In [44]:
start = time.time()
print n in python_list
duration = time.time() - start
print "%f Seconds" % duration

False
0.683922 Seconds


Tuple - static array

In [45]:
start = time.time()
print n in python_tuple
duration = time.time() - start
print "%f Seconds" % duration

False
0.366091 Seconds


Tuples are cached by Python; therefore, performance may significantly improve after the first time a tuple is called.

In [46]:
# Second Run
start = time.time()
print n in python_tuple
duration = time.time() - start
print "%f Seconds" % duration

False
0.390532 Seconds


Set.  

A set (or dictionary) is ideal data structure if your data does not have intrinsic order and have unique elements.

In [47]:
start = time.time()
print n in python_set
duration = time.time() - start
print "%f Seconds" % duration

False
0.000222 Seconds


### 1.2.2. Iteration

List

In [48]:
start = time.time()
for i in python_list:
    i
duration = time.time() - start
print "%f Seconds" % duration

1.592714 Seconds


Tuple

In [49]:
start = time.time()
for i in python_tuple:
    i
duration = time.time() - start
print "%f Seconds" % duration

0.919640 Seconds


Set

In [50]:
start = time.time()
for i in python_set:
    i
duration = time.time() - start
print "%f Seconds" % duration

1.602380 Seconds


### 1.3. List Comprehension vs For Loop

For Loop

In [51]:
start = time.time()
list_forloop = list()
for i in xrange(n):
    list_forloop.append(i)
duration = time.time() - start
print "%f Seconds" % duration

2.328954 Seconds


List Comprehension

In [52]:
start = time.time()
list_comprehension = [i for i in xrange(n)]
duration = time.time() - start
print "%f Seconds" % duration

1.121428 Seconds


### 1.4. Mutability

### 1.4.1. Comparing List vs Tuple

In [53]:
my_list = ['a','b','c','d']
my_tuple = ('a','b','c','d')

In [54]:
my_list[0] = 'A'
my_list

['A', 'b', 'c', 'd']

Let's attempt to change the first element of my_tuple.

In [55]:
my_tuple[0] = 'A'
my_tuple

TypeError: 'tuple' object does not support item assignment

### 1.4.2. Copying Lists

In [ ]:
my_list = ['a','b','c','d']

In [ ]:
my_list2 = my_list

Let's change the second element of my_list2

In [ ]:
my_list2[1] = 'B'
my_list2

We ended up changing the second element in my_list to 'B'.

In [ ]:
my_list

To avoid this problem, there are three ways to copy a list that will leave the original list unperturbed. 

###### Make element-wise copy to a new list

In [ ]:
my_list = ['a','b','c','d']
my_list2 = my_list[:]
my_list2[1] = 'B'
print "my_list:", my_list, "| my_list2:", my_list2

##### Create a new list out of the original list

In [ ]:
my_list = ['a','b','c','d']
my_list2 = list(my_list)
my_list2[1] = 'B'
print "my_list:", my_list, "| my_list2:", my_list2

###### Import copy module and make a deepcopy

In [ ]:
my_list = ['a','b','c','d']
import copy
my_list2 = copy.deepcopy(my_list)
my_list2[1] = 'B'
print "my_list:", my_list, "| my_list2:", my_list2

### 1.4. Line Profiling

In [ ]:
import line_profiler
import IPython
ip = IPython.get_ipython()
ip.define_magic('lprun', line_profiler.magic_lprun)

Let's use the primes function that we used earlier to demonstrate what line-profiler can do for us.

In [ ]:
def primes(n): 
    if n==2:
        return [2]
    elif n<2:
        return []
    s=range(3,n+1,2)
    mroot = n ** 0.5
    half=(n+1)/2-1
    i=0
    m=3
    while m <= mroot:
        if s[i]:
            j=(m*m-3)/2
            s[j]=0
            while j<half:
                s[j]=0
                j+=m
        i=i+1
        m=2*i+3
    return [2]+[x for x in s if x]

In [ ]:
%lprun -f primes primes(100)

For detailed discussions about optimizing performance in Python, checkout [High Performance Python](http://shop.oreilly.com/product/0636920028963.do)
<img src="img/lrg.jpg" style="max-width: 200px;">

## 2. NetworkX

NetworkX is a popular Python library used to model networks including social network graphs.  Using Networkx, we can model both undirected and directed graphs.

For the Influence Maximization Problem, we will provide you with a pre-built graph of Yelp Reviewer network and ask you to use Networkx to implement the Independent Cascade function. 

### 2.1. Installation

If you installed Python through Anaconda distribution, you should already have NetworkX in your Python environment.  

If you do not have it, you may run the following command below.

In [ ]:
import networkx as nx

In [ ]:
nx.__version__

You should use NetworkX version 1.8 or above.

### 2.2. Examples

Creating a sample graph

In [ ]:
random_graph = nx.erdos_renyi_graph(n=20,p=0.6,seed=42)

Finding nodes

In [ ]:
random_graph.nodes()

In [ ]:
random_graph[12]

Finding edges

In [ ]:
random_graph.edges()

In [ ]:
random_graph.edge[0][3]

Storing edge weights

In [ ]:
for s,t in random_graph.edges_iter():
    random_graph[s][t]['weight'] = np.random.uniform()

In [ ]:
random_graph.edge[0][3]

For more examples, please explore the [NetworkX documentation](http://networkx.readthedocs.org/en/stable/index.html).

### 2.3. Exploring Yelp Review Network 

Before we scale up the Influence Maximization Problem on larger network, we will work with a "toy" network that is a sample of North Carolina reviewer network.  This network contains 240 nodes and 920 edges.

By week 3, you will solve the IMP on this smaller network, and by week 4, you will implement your algorithms on the larger network, which contains over 350 thousand nodes and 4 million edges.

In [ ]:
import json
from networkx.readwrite import json_graph

In [ ]:
with open("graph/nc_mini.json", "r") as graph_data:
    graph_data = json.load(graph_data)
    NC_digraph = json_graph.node_link_graph(graph_data)

In [ ]:
NC_digraph.edges(data=True)

In [ ]:
NC_digraph.nodes('PpkKVodWC0sdn74TbHQLzA')

In [ ]:
print NC_digraph.number_of_nodes(), "nodes"

In [ ]:
print NC_digraph.number_of_edges(), "edges"

In [ ]:
NC_digraph.succ['Z1FWaNNO8oxaHLYB9XhQDg']

### 2.2. Visualizing the Graph

In [ ]:
def print_graph(Graph, S1=None):
    plt.figure(figsize=(16,10))
    color_map = {1: 'b', 0: 'r'}
    pos = nx.random_layout(Graph)
    
    if S1:
        nx.draw_networkx(Graph, pos, with_labels=False, node_size=100, node_shape='.',
                linewidth=None, width=0.2, edge_color='y', 
                node_color=[color_map[Graph.node[node]['action']] for node in Graph],
                edgelist=reduce(lambda x,y: x+y,[Graph.edges(node) for node in S1]))
        nx.draw_networkx_nodes(Graph, pos, nodelist=S1, node_color="b", node_size=150, 
                              node_shape="*", label="Initial Set")
        plt.legend()
    else:
        nx.draw_networkx(Graph, pos, with_labels=False, node_size=100, node_shape='.',
                linewidth=None, width=0.2, edge_color='y', 
                 node_color=[color_map[Graph.node[node]['action']] for node in Graph])
        
    plt.xlim(-0.05,1.05)
    plt.ylim(-0.05,1.05)
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [ ]:
print_graph(NC_digraph)

In [ ]:
NC_digraph


In [ ]:
#NC_digraph.node['uBUGZtTxmaG-8YpUWpU5_Q']['action']
#NC_digraph.add_node('uBUGZtTxmaG-8YpUWpU5_Q', action = 0)
print NC_digraph.node.keys()

In [ ]:
NC_digraph.node['GtcVim7Y43ALraAb3ritmQ']['action'] = 0
NC_digraph.node['GtcVim7Y43ALraAb3ritmQ']['action']

In [ ]:
def reset():
    for i in NC_digraph:
        NC_digraph.node[i]['action'] = 0

In [ ]:
np.random.seed(24)
init_nodes = np.random.choice(NC_digraph.nodes(), 1)[0]

In [ ]:
init_nodes

In [ ]:
def cascade(init_nodes):
    NC_digraph.node[init_nodes]['action'] = 1
    init_list = []
    init_list.append(init_nodes)
    while len(init_list) != 0 :
        curr_node = init_list.pop(0)
        #curr_node = init_list[0]
        #init_list = init_list[1:]
        for i in NC_digraph[curr_node]:
            if NC_digraph.node[i]['action'] == 0:
                b = NC_digraph.node[i]['review_count']
                a =  NC_digraph[curr_node][i]['weight']
                #np.random.seed(9)
                b_dist = np.sqrt(np.random.beta(a = a, b = b))
                #np.random.seed(9)
                u_dist = np.random.uniform(0,1)
                if b_dist > u_dist:
                    #print init_list,'hi'
                    #NC_digraph.add_node(i, action = 1)
                    NC_digraph.node[i]["action"] = 1
                    init_list.append(i)
                    #print init_list,'bye'
                #else:
                 #   NC_digraph.node[i]['action'] = 2

In [31]:
def count():
    n = sum([i[1]['action'] for i in NC_digraph.nodes(data = True)])
    return n

  
#print NC_digraph.nodes(data=True)['PpkKVodWC0sdn74TbHQLzA']
#NC_digraph.edges(data=True)
#NC_digraph['PpkKVodWC0sdn74TbHQLzA']['S3HZF5aANmhZoMkFkPMdqQ']['weight']

#type(NC_digraph.nodes(data=True))
#nx.get_node_attributes(NC_digraph, 'review_count')['-_1ctLaz3jhPYc12hKXsEQ']

In [32]:
NC_digraph.nodes(data = True)

NameError: name 'NC_digraph' is not defined

In [33]:
reset()
cascade(init_nodes)
print count()

          

NameError: name 'reset' is not defined

In [26]:
%lprun -f count count() #N = 100 --- 71.6238 s

ERROR: Line magic function `%lprun` not found.


In [ ]:
NC_digraph.node

In [ ]:
np.random.beta?

In [ ]:
NC_digraph.nodes(data=True)

In [ ]:
def b_test(N):
    result = []
    #start_time  = time.time()
#     np.random.seed(24)
    for i in range(N):
        reset()
        cascade(init_nodes)
        result.append(count())
    #return {'mean':np.mean(result), 'var':np.std(result)}
 

In [30]:
%lprun -f b_test b_test(1000)


UsageError: Could not find function u'b_test'.
NameError: name 'b_test' is not defined

In [ ]:
%lprun -f b_test b_test(24) #N = 100 --- 71.6238 s

In [ ]:
#N: number of runs for an influence estimate I^

def c_test(root_seed = 24):
    result = []
    for i in range(100):
        tmp = b_test(100, root_seed)
        result.append(tmp['mean'])
    return {'mean':np.mean(result), 'std': np.std(result)}

def test_plot(sample_list):
    sample_mean = []
    sample_std = []
    for i in sample_list:
        sample_mean.append(c_test(i)['mean'])
        sample_std.append(np.std(sample_mean[i]))
    plt.plot(sample_list, sample_mean, label = 'MEAN')
    plt.plot(sample_list, sample_std, label = 'STD')
    print 'mean', sample_mean
    print 'std', sample_std
    plt.legend()


In [ ]:
test_plot([100, 300, 1000])

In [ ]:
c_test(24)

In [ ]:
print 1